In [30]:
!pip install mlflow

In [38]:
!pip install mlflow-skinny sqlalchemy alembic sqlparse

In [35]:
import os 
from random import random, randint 
from mlflow import log_metric, log_param, log_artifacts
from mlflow.tensorflow import TensorSpec

In [39]:
mlflow.set_tracking_uri("sqlite:///my.db")

In [46]:
import os
from random import random, randint
from mlflow import log_metric, log_param, log_artifacts

if __name__ == "__main__":
    # Log a parameter (key-value pair)
    log_param("param_", randint(0, 100))

    # Log a metric; metrics can be updated throughout the run
    log_metric("foo", random())
    log_metric("foo", random() + 1)
    log_metric("foo", random() + 2)

    # Log an artifact (output file)
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("hello world!")
    log_artifacts("outputs")


MlflowException: Run with id=4de02ef21ade49569221018bc08b5345 not found

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [5]:
import mlflow

In [6]:
mlflow.set_experiment("linear_regression")

<Experiment: artifact_location='file:///root/zigma/mlruns/598271008479588814', creation_time=1679654029883, experiment_id='598271008479588814', last_update_time=1679654029883, lifecycle_stage='active', name='linear_regression', tags={}>

In [7]:
df=pd.read_csv("DailyDelhiClimateTest.csv")

In [8]:
df

,date,meantemp,humidity,wind_speed,meanpressure
0,2017-01-01,15.913043,85.869565,2.743478,59.000000
1,2017-01-02,18.500000,77.222222,2.894444,1018.277778
2,2017-01-03,17.111111,81.888889,4.016667,1018.333333
3,2017-01-04,18.700000,70.050000,4.545000,1015.700000
4,2017-01-05,18.388889,74.944444,3.300000,1014.333333
...,...,...,...,...,...
109,2017-04-20,34.500000,27.500000,5.562500,998.625000
110,2017-04-21,34.250000,39.375000,6.962500,999.875000
111,2017-04-22,32.900000,40.900000,8.890000,1001.600000
112,2017-04-23,32.875000,27.500000,9.962500,1002.125000


In [9]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 114 entries, 2017-01-01 to 2017-04-24
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   meantemp      114 non-null    float64
 1   humidity      114 non-null    float64
 2   wind_speed    114 non-null    float64
 3   meanpressure  114 non-null    float64
dtypes: float64(4)
memory usage: 4.5 KB


In [11]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

In [12]:
df

,meantemp,humidity,wind_speed,meanpressure,day,month,year
date,,,,,,,
2017-01-01,15.913043,85.869565,2.743478,59.000000,1,1,2017
2017-01-02,18.500000,77.222222,2.894444,1018.277778,2,1,2017
2017-01-03,17.111111,81.888889,4.016667,1018.333333,3,1,2017
2017-01-04,18.700000,70.050000,4.545000,1015.700000,4,1,2017
2017-01-05,18.388889,74.944444,3.300000,1014.333333,5,1,2017
...,...,...,...,...,...,...,...
2017-04-20,34.500000,27.500000,5.562500,998.625000,20,4,2017
2017-04-21,34.250000,39.375000,6.962500,999.875000,21,4,2017
2017-04-22,32.900000,40.900000,8.890000,1001.600000,22,4,2017


In [13]:
df = df.reindex(columns=["day","month","year","humidity","wind_speed","meanpressure","meantemp"])

In [14]:
df

,day,month,year,humidity,wind_speed,meanpressure,meantemp
date,,,,,,,
2017-01-01,1,1,2017,85.869565,2.743478,59.000000,15.913043
2017-01-02,2,1,2017,77.222222,2.894444,1018.277778,18.500000
2017-01-03,3,1,2017,81.888889,4.016667,1018.333333,17.111111
2017-01-04,4,1,2017,70.050000,4.545000,1015.700000,18.700000
2017-01-05,5,1,2017,74.944444,3.300000,1014.333333,18.388889
...,...,...,...,...,...,...,...
2017-04-20,20,4,2017,27.500000,5.562500,998.625000,34.500000
2017-04-21,21,4,2017,39.375000,6.962500,999.875000,34.250000
2017-04-22,22,4,2017,40.900000,8.890000,1001.600000,32.900000


In [15]:
x=df.iloc[:,:6]
y=df.iloc[:,-1]

In [16]:
x

,day,month,year,humidity,wind_speed,meanpressure
date,,,,,,
2017-01-01,1,1,2017,85.869565,2.743478,59.000000
2017-01-02,2,1,2017,77.222222,2.894444,1018.277778
2017-01-03,3,1,2017,81.888889,4.016667,1018.333333
2017-01-04,4,1,2017,70.050000,4.545000,1015.700000
2017-01-05,5,1,2017,74.944444,3.300000,1014.333333
...,...,...,...,...,...,...
2017-04-20,20,4,2017,27.500000,5.562500,998.625000
2017-04-21,21,4,2017,39.375000,6.962500,999.875000
2017-04-22,22,4,2017,40.900000,8.890000,1001.600000


In [17]:
y

date
2017-01-01    15.913043
2017-01-02    18.500000
2017-01-03    17.111111
2017-01-04    18.700000
2017-01-05    18.388889
                ...    
2017-04-20    34.500000
2017-04-21    34.250000
2017-04-22    32.900000
2017-04-23    32.875000
2017-04-24    32.000000
Name: meantemp, Length: 114, dtype: float64

In [18]:
model = LinearRegression()
model.fit(x, y)

LinearRegression()

In [21]:
y_pred = model.predict(x)
mse = mean_squared_error(y, y_pred)

In [ ]:
#mlflow.projects.run(".", entry_point="train.py", use_conda=False)

In [26]:
mlflow.projects.run(".", entry_point="Delhi dataset(ML flow).py")

2023/03/28 11:04:56 WARNING mlflow.tracking._tracking_service.utils: Failed to import Git (the git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2023/03/28 11:04:56 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad 

UnsupportedModelRegistryStoreURIException:  Model registry functionality is unavailable; got unsupported URI 'mlflow-artifacts:/0' for model registry data storage. Supported URI schemes are: ['', 'file', 'databricks', 'databricks-uc', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.

In [ ]:
!pip install optuna


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    max_depth = trial.suggest_int("max_depth", 3, 20)

    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    score = cross_val_score(clf, x_train, y_train, cv=5).mean()

    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

best_params = study.best_params

In [25]:
import mlflow

mlflow.set_tracking_uri("mlflow-artifacts:/0")
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

Current tracking uri: mlflow-artifacts:/0
